<a href="https://colab.research.google.com/github/kolomichyk/BMSTU-Hack/blob/develop/Po_sqrt_team_(1)_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
from seaborn import pairplot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from statistics import mean
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

###Предобработка данных

In [8]:
data_target = pd.read_csv('/content/train_target.csv')

In [10]:
data = pd.read_parquet('/content/train.parquet')

In [11]:
def addAnswer(data):
  for j in range(0, data.shape[0], 19):
    NotNullElement  = False
    posNotNullElement = 0
    for i in range(j+18, j-1, -1):
      if (data['milleage_all'][i] and not NotNullElement):
        NotNullElement = True
        posNotNullElement = i
      if (data['milleage_all'][i]):
        data['answer'][i] = data_target['target'][j//19] +  data['milleage_all'][posNotNullElement] - data['milleage_all'][i]

Количество пропусков в данных

In [ ]:
data['answer'] = [0 for i in range(data.shape[0])]

In [ ]:
addAnswer(data)

<ipython-input-23-a71db577cb51>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['answer'][i] = data_target['target'][j//19] +  data['milleage_all'][posNotNullElement] - data['milleage_all'][i]


Удаляем строки с пропусками

In [ ]:
df = data
df = df.dropna(axis=0)
df = df.drop(columns=['wagnum', 'ts_id'])
df.shape

(620886, 10)

In [ ]:
df.isnull().sum()

milleage_all       0
axl1_l_w_flange    0
axl1_r_w_flange    0
axl2_l_w_flange    0
axl2_r_w_flange    0
axl3_l_w_flange    0
axl3_r_w_flange    0
axl4_l_w_flange    0
axl4_r_w_flange    0
answer             0
dtype: int64

Нормализуем данные

In [ ]:
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

### Визуализируем данные

In [ ]:
vdata1 = df_normalized[:1000]

In [ ]:
pairplot(vdata1)

In [ ]:
vdata1 = vdata1.assign(avg1=vdata1[['axl1_l_w_flange', 'axl1_r_w_flange']].mean(axis=1))
vdata1 = vdata1.assign(avg2=vdata1[['axl2_l_w_flange', 'axl2_r_w_flange']].mean(axis=1))
vdata1 = vdata1.assign(avg3=vdata1[['axl3_l_w_flange', 'axl3_r_w_flange']].mean(axis=1))
vdata1 = vdata1.assign(avg4=vdata1[['axl4_l_w_flange', 'axl4_r_w_flange']].mean(axis=1))

In [ ]:
pairplot(vdata1.drop(columns=['axl1_l_w_flange', 'axl1_r_w_flange', 'axl2_l_w_flange',
       'axl2_r_w_flange', 'axl3_l_w_flange', 'axl3_r_w_flange',
       'axl4_l_w_flange', 'axl4_r_w_flange']))

### Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_normalized.drop(columns=['answer']), df['answer'], test_size=0.20, random_state=42)

### Построение MVP модели

In [ ]:
linear_model = LinearRegression().fit(X_train, y_train)

In [ ]:
linear_model.score(X_test, y_test)

0.30000308810642407

In [ ]:
linear_model.coef_

array([2503.8054303 , 3616.34705899, 3419.74625692, 4271.59964096,
       3283.19374453, 3178.51222309, 4354.90039529, 2946.96618875,
       3692.85747515])

In [ ]:
lassoModel = Lasso(alpha = i * 0.25, tol = 0.0925)
for i in range(1, 9):
    lassoModel.fit(X_train, y_train)
    scores = cross_val_score(lassoModel, X_train, y_train, cv = 10)
    avg_cross_val_score = mean(scores)*100

In [ ]:
lassoModel.score(X_test, y_test)

0.3000033264738413

In [ ]:
lassoModel.coef_

array([2500.46019163, 3616.86815621, 3418.48137326, 4268.77719003,
       3280.05339539, 3176.46754717, 4354.25616113, 2947.88565021,
       3694.55949233])

In [ ]:
avg_cross_val_score

30.131320657597914

In [ ]:
mean_absolute_error(y_test, lassoModel.predict(X_test))

23848.71555115389

In [ ]:
mean_absolute_error(y_test, linear_model.predict(X_test))

23848.742280832463

### RandomForestRegressor

In [ ]:
Forest_regressor = RandomForestRegressor(100, criterion="absolute_error", max_depth=15)

In [ ]:
for epoch in range(1):
  Forest_regressor.fit(X_train, y_train)

In [ ]:
mean_absolute_error(y_test, Forest_regressor.predict(X_test))

### XGBoost

In [ ]:
xgb_r = xgb.XGBRegressor(objective ='reg:linear',
                  n_estimators = 10, seed = 42)

In [ ]:
xgb_r.fit(X_train, y_train)

[05:50:14] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [ ]:
mean_absolute_error(y_test, xgb_r.predict(X_test))

20146.426607667923

### Вторая теория (Усредним значения)

In [ ]:
data.head(20)

In [13]:
df = pd.DataFrame(index=range(data.shape[0]//19),columns=data.columns)

In [14]:
data['answer'] = [0 for i in range(data.shape[0])]

In [15]:
def make_data(data):
  df = pd.DataFrame(index=range(data.shape[0]//19),columns=data.columns)
  for j in range(0, data.shape[0], 19):
    milleage_all_avg = 0
    axl1_l_w_flange_avg = 0
    axl1_r_w_flange_avg = 0
    axl2_l_w_flange_avg = 0
    axl2_r_w_flange_avg = 0
    axl3_l_w_flange_avg = 0
    axl3_r_w_flange_avg = 0
    axl4_l_w_flange_avg = 0
    axl4_l_w_flange_avg = 0
    axl4_r_w_flange_avg = 0
    milleage_all_kol = 0
    axl1_l_w_flange_kol = 0
    axl1_r_w_flange_kol = 0
    axl2_l_w_flange_kol = 0
    axl2_r_w_flange_kol = 0
    axl3_l_w_flange_kol = 0
    axl3_r_w_flange_kol = 0
    axl4_l_w_flange_kol = 0
    axl4_l_w_flange_kol = 0
    axl4_r_w_flange_kol = 0
    for i in range(j+18, j-1, -1):
      if data['milleage_all'][i]:
        milleage_all_avg += data['milleage_all'][i]
        milleage_all_kol += 1
      if data['axl1_l_w_flange'][i]:
        axl1_l_w_flange_avg += data['axl1_l_w_flange'][i]
        axl1_l_w_flange_kol += 1
      if data['axl1_r_w_flange'][i]:
        axl1_r_w_flange_avg += data['axl1_r_w_flange'][i]
        axl1_r_w_flange_kol += 1 
      if data['axl2_l_w_flange'][i]:
        axl2_l_w_flange_avg += data['axl2_l_w_flange'][i]
        axl2_l_w_flange_kol += 1
      if data['axl2_r_w_flange'][i]:
        axl2_r_w_flange_avg += data['axl2_r_w_flange'][i]
        axl2_r_w_flange_kol += 1 
      if data['axl3_l_w_flange'][i]:
        axl3_l_w_flange_avg += data['axl3_l_w_flange'][i]
        axl3_l_w_flange_kol += 1
      if data['axl3_r_w_flange'][i]:
        axl3_r_w_flange_avg += data['axl3_r_w_flange'][i]
        axl3_r_w_flange_kol += 1 
      if data['axl4_l_w_flange'][i]:
        axl4_l_w_flange_avg += data['axl4_l_w_flange'][i]
        axl4_l_w_flange_kol += 1
      if data['axl4_r_w_flange'][i]:
        axl4_r_w_flange_avg += data['axl4_r_w_flange'][i]
        axl4_r_w_flange_kol += 1 
    df['milleage_all'][j//19] = milleage_all_avg / milleage_all_kol
    df['axl1_l_w_flange'][j//19] = axl1_l_w_flange_avg / axl1_l_w_flange_kol
    df['axl1_r_w_flange'][j//19] = axl1_r_w_flange_avg / axl1_r_w_flange_kol
    df['axl2_l_w_flange'][j//19] = axl2_l_w_flange_avg / axl2_l_w_flange_kol
    df['axl2_r_w_flange'][j//19] = axl2_r_w_flange_avg / axl2_r_w_flange_kol
    df['axl3_l_w_flange'][j//19] = axl3_l_w_flange_avg / axl3_l_w_flange_kol
    df['axl3_r_w_flange'][j//19] = axl3_r_w_flange_avg / axl3_r_w_flange_kol
    df['axl4_l_w_flange'][j//19] = axl4_l_w_flange_avg / axl4_l_w_flange_kol
    df['axl4_r_w_flange'][j//19] = axl4_r_w_flange_avg / axl4_r_w_flange_kol
  return df

In [16]:
df = make_data(data)

In [17]:
df.head()

,wagnum,ts_id,milleage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,answer
0,NaN,NaN,48215.157895,28.626316,28.852632,28.347368,28.552632,29.668421,29.426316,29.584211,27.805263,NaN
1,NaN,NaN,18275.736842,29.884211,31.868421,31.747368,32.042105,29.984211,31.789474,31.657895,30.973684,NaN
2,NaN,NaN,28330.315789,28.805263,27.868421,28.247368,27.189474,28.436842,28.873684,28.884211,28.173684,NaN
3,NaN,NaN,19096.157895,29.926316,27.752632,27.984211,29.178947,30.778947,29.1,30.505263,28.421053,NaN
4,NaN,NaN,11969.263158,32.173684,32.742105,32.384211,28.926316,32.484211,32.715789,31.742105,31.336842,NaN


In [18]:
y = pd.Series(data_target['target'])

In [19]:
df = df.drop(columns=['wagnum', 'ts_id'])

In [ ]:
df.head()

,milleage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange
0,48215.157895,28.626316,28.852632,28.347368,28.552632,29.668421,29.426316,29.584211,27.805263
1,18275.736842,29.884211,31.868421,31.747368,32.042105,29.984211,31.789474,31.657895,30.973684
2,28330.315789,28.805263,27.868421,28.247368,27.189474,28.436842,28.873684,28.884211,28.173684
3,19096.157895,29.926316,27.752632,27.984211,29.178947,30.778947,29.1,30.505263,28.421053
4,11969.263158,32.173684,32.742105,32.384211,28.926316,32.484211,32.715789,31.742105,31.336842


In [20]:
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

/usr/local/lib/python3.9/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.9/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.9/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [ ]:
df_normalized.head()

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df_normalized, y, test_size=0.10, random_state=42)

In [23]:
xgb_test_r = xgb.XGBRegressor(objective ='reg:linear',
                  n_estimators = 28, seed = 42)

In [24]:
xgb_test_r.fit(X_train, y_train)

[14:49:30] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=28, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [25]:
mean_absolute_error(y_test, xgb_test_r.predict(X_test))

19163.13172200028

In [26]:
import pickle

In [28]:
with open('/content/Model.pkl', 'wb') as f:
    pickle.dump(xgb_test_r, f)

In [29]:
with open('/content/Model.pkl', 'rb') as f:
    t1 = pickle.load(f)

In [30]:
mean_absolute_error(y_test, t1.predict(X_test))

19163.13172200028